In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import gc, glob, os, datetime, dateutil
import datetime

In [2]:
#This code reads the class information data. 
#For every class, there is a class location and info on when the class occurs
class_info = pd.read_json('class_info.json')

In [3]:
#This code extracts the location row of the class_info dataset to create a list of class locations.
all_locations = list(class_info.iloc[0,:])

In [4]:
#Since some classes have the same locations, some locations appear multiple times. This code finds only the unique class locations. 
def unique(list1):
    unique_locations = []
    for i in list1:
        if i not in unique_locations:
            unique_locations.append(i)
    return list(unique_locations)
locations = unique(all_locations)

In [5]:
#To match the location data in the wifi location dataset, the locations were formatted in a certain way. 
#This code edits the list of unique class locations to match that formatting for further analysis.
def location(list2):
    class_loc = []
    for i in list2:
        class_loc.append('in[' + str(i) + ']')
    return class_loc
class_locations = location(locations)

In [6]:
#This code reads the wifi location dataset.
wifi_locations = pd.read_csv("wifi_location.csv")

#This code returns the rows of the dataset that match classroom locations.
def only_class_locations(data):
    df_with_only_class_locations = data[data.location.isin(class_locations)] 
    return df_with_only_class_locations
wifi_locations = only_class_locations(wifi_locations)

#The following two lines of code convert timestamp data the pandas datetime format and then a new column is created to find the day of the year for each timestamp. 
def time_day_of_location_timestamps(data):
    wifi_locations['time'] = pd.to_datetime(data['time'])
    wifi_locations['day'] = data['time'].dt.dayofyear
    wifi_locations['index'] = wifi_locations.index
    return wifi_locations
wifi_locations = time_day_of_location_timestamps(wifi_locations)

#To loop through the code later, I tried to reset the index of the dataframe since the original indices were not uniform.
#I saved the original indices to a new column titled "index"
wifi_locations['index'] = wifi_locations.index
wifi_locations = wifi_locations.reset_index()
wifi_locations

,level_0,timestamp,location,uid,time,day,index
0,0,1364396255,in[sudikoff],u45,2013-03-27 14:57:35,86,0
1,1,1364397473,in[sudikoff],u45,2013-03-27 15:17:53,86,1
2,2,1364397623,in[sudikoff],u45,2013-03-27 15:20:23,86,2
3,3,1364397661,in[sudikoff],u45,2013-03-27 15:21:01,86,3
4,4,1364397671,in[sudikoff],u45,2013-03-27 15:21:11,86,4
...,...,...,...,...,...,...,...
549125,1893833,1369815184,in[lsb],u31,2013-05-29 08:13:04,149,1893833
549126,1893834,1369815204,in[lsb],u31,2013-05-29 08:13:24,149,1893834
549127,1893835,1369815224,in[lsb],u31,2013-05-29 08:13:44,149,1893835
549128,1893836,1369815244,in[lsb],u31,2013-05-29 08:14:04,149,1893836


In [7]:
#To find the time delta for each location, I used this function.
#The shift number represents the difference in the index values of the rows where the location first appeared and when the location last continously appeared.
#To calculate the shift number, I constrained it to locations that appeared consecutively in the same data since some locations appeared continously across multiple days.

def shift_counter(data):
    shift_num = 0
    #counter = 0
    list_shift_num = []
    list_id = []
    for i in range(0, len(data)):
        #if data.location[i] == data.location[i+1]:
        try: 
            if data.location[i] == data.location[i+1] and data.day[i] == data.day[i+1]:
            #data.index[i] == (data.index[i+1] - 1)
                shift_num += 1
                #uid = data.uid[i]
            else:
                list_shift_num.append(shift_num)
                shift_num = 0
                #list_id.append(uid)
        except:
            list_shift_num.append(shift_num)
            shift_num = 0
            #list_id.append(uid)
    return list_shift_num

#This list has all of the shift numbers (# of consecutive rows where location and day are the same)
list_shift = shift_counter(wifi_locations)
list_shift

[4,
 2,
 4,
 1,
 0,
 2,
 1,
 8,
 9,
 48,
 4,
 0,
 4,
 8,
 2,
 0,
 351,
 2,
 372,
 9,
 15,
 0,
 2,
 369,
 48,
 2,
 7,
 1,
 0,
 6,
 0,
 1,
 1,
 274,
 7,
 0,
 12,
 1,
 1,
 3,
 38,
 3,
 1,
 2,
 1,
 18,
 0,
 0,
 0,
 0,
 1,
 1,
 4,
 0,
 12,
 1,
 0,
 202,
 528,
 8,
 4,
 3,
 1,
 31,
 1,
 1,
 3,
 2,
 0,
 9,
 0,
 35,
 0,
 0,
 21,
 0,
 7,
 1,
 162,
 0,
 19,
 4,
 195,
 2,
 0,
 46,
 0,
 17,
 0,
 54,
 18,
 0,
 0,
 0,
 1,
 259,
 0,
 0,
 9,
 193,
 0,
 184,
 3,
 1,
 10,
 0,
 0,
 248,
 180,
 1,
 715,
 109,
 0,
 4,
 279,
 3,
 2,
 1,
 4,
 1,
 12,
 1,
 0,
 0,
 52,
 3,
 14,
 3,
 1,
 5,
 0,
 2,
 0,
 2,
 1,
 3,
 1,
 1,
 1,
 8,
 24,
 6,
 0,
 1,
 15,
 26,
 0,
 0,
 17,
 32,
 0,
 3,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 37,
 119,
 0,
 0,
 0,
 6,
 30,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 2,
 5,
 7,
 1,
 6,
 0,
 0,
 16,
 0,
 14,
 0,
 0,
 2,
 0,
 1,
 7,
 7,
 4,
 1,
 61,
 1,
 1,
 2,
 1,
 0,
 12,
 0,
 4,
 128,
 0,
 0,
 0,
 0,
 16,
 4,
 4,
 1,
 1,
 51,
 94,
 0,
 15,
 1,
 2,
 32,
 3,
 3,
 2,
 1,
 0,
 4,
 0,
 5,
 34,
 4,
 0,
 1,


In [8]:
#This function is the same as the previous function.
#The only difference is that I tracked the start timestamps, user id, and location associated with shift number.

def shift_counter_id(data):
    shift_num = 0
    #counter = 0
    list_shift_num = []
    list_id = []
    list_time = []
    list_location = []
    for i in range(0, len(data)):
        #if data.location[i] == data.location[i+1]:
        try: 
            if data.location[i] == data.location[i+1] and data.day[i] == data.day[i+1]:
            #data.index[i] == (data.index[i+1] - 1)
                shift_num += 1
                uid = data.uid[i]
                time = data.timestamp[i]
                location = data.location[i]
            else:
                list_shift_num.append(shift_num)
                shift_num = 0
                list_id.append(uid)
                list_time.append(time)
                list_location.append(location)
        except:
            list_shift_num.append(shift_num)
            shift_num = 0
            list_id.append(uid)
            list_time.append(time)
            list_location.append(location)
    df = pd.DataFrame({'Shift Num': list_shift_num, 'uid': list_id, 'time': list_time, 'location': list_location}) #'time': list_time, 'location': list_location})
    return df

#This dataframe contains the shift numbers and their associated times, locations and user ids
dataframe = shift_counter_id(wifi_locations)

In [9]:
dataframe

,Shift Num,uid,time,location
0,4,u45,1364397661,in[sudikoff]
1,2,u45,1364839542,in[sudikoff]
2,4,u45,1364842439,in[lsb]
3,1,u45,1364848920,in[baker-berry]
4,0,u45,1364848920,in[baker-berry]
...,...,...,...,...
16143,1,u31,1369317608,in[baker-berry]
16144,7,u31,1369324215,in[steele]
16145,171,u31,1369329059,in[sudikoff]
16146,288,u31,1369438492,in[sudikoff]


In [10]:
#Since there were some shift numbers that were zero and since these shift numbers will not result in a time delta, I took them out. 
#I created a new list of non-zero shift numbers

def shifts_only(list1):
    shifts_only_list = []
    for i in list1:
        if i != 0:
            shifts_only_list.append(i)
    return shifts_only_list

#This list is a list of non-zero shift numbers.
list_shifts = shifts_only(list_shift)
#list_shifts

In [11]:
#When calculating the time delta, I would need the index of when the location first appeared and the index when the location last consecutively appeared. 
#I had to find the formula to find the start and ending indices, and it involved, for every element in the list of shift number, the sum of the shift numbers before it.
def get_sums(list1):
    list_sums_b = []
    for i in range(0,len(list1)+1):
        new_list = list1[:i]
        sums = sum(new_list)
        list_sums_b.append(sums)
    return list_sums_b

#This function returns a list in which every element represents the sum of the shift numbers that came before it.
list_sums_before = get_sums(list_shifts)

In [12]:
#This function calculates the time deltas for every location. 
def time_delta(list_shift_num):
    time_deltas = []
    for i in range(0, len(list_shift_num)):
        if i == 0:
            time_deltas.append(wifi_locations['time'][list_shift_num[i]] - wifi_locations['time'][0])
        elif i != 0:
            #To find the time deltas for all the elements after the first one, I used the following formula:
            time_deltas.append(wifi_locations['time'][list_shift_num[i]+i+list_sums_before[i]] - wifi_locations['time'][list_sums_before[i]+i])
    return time_deltas

#This function returns the time deltas.
#time_delta(list_shifts)

In [13]:
#This function is the exact same as the previous function but returns a dataframe instead.
#The dataframe consists of the time deltas and their associated time, location, and user id.
def time_delta(data, list_shift_num):
    time_deltas = []
    location = []
    start_time = []
    uid = []
    for i in range(0, len(list_shift_num)):
        if i == 0:
            time_deltas.append(wifi_locations['time'][list_shift_num[i]] - wifi_locations['time'][0])
            uid.append(data.uid[i])
            start_time.append(data.time[i])
            location.append(data.location[i])
        elif i != 0:
            time_deltas.append(wifi_locations['time'][list_shift_num[i]+i+list_sums_before[i]] - wifi_locations['time'][list_sums_before[i]+i])
            uid.append(data.uid[i])
            start_time.append(data.time[i])
            location.append(data.location[i])
        df1 = pd.DataFrame({'Time Delta': time_deltas, 'uid': uid, 'start time': start_time, 'location': location})
    return df1

#The function returns the new dataframe.
variable = time_delta(dataframe, list_shifts)

In [14]:
#For convenvience, I redefined the dataframe as "df"
df = variable

In [15]:
#This code returned two new columns with the time deltas in seconds and minutes.
#I converted the time deltas to minutes for convenience when later trying to see if the student actually attended class or not based off of how much time they spent in the classroom.
df['Time Delta (in seconds)'] = df['Time Delta'].dt.total_seconds()
df['Time Delta (in minutes)'] = df['Time Delta (in seconds)']/60
df['start time in regular notation'] = pd.to_datetime(df['start time'], unit = 's')
df

,Time Delta,uid,start time,location,Time Delta (in seconds),Time Delta (in minutes),start time in regular notation
0,00:23:36,u45,1364397661,in[sudikoff],1416.0,23.600000,2013-03-27 15:21:01
1,00:20:58,u45,1364839542,in[sudikoff],1258.0,20.966667,2013-04-01 18:05:42
2,00:03:44,u45,1364842439,in[lsb],224.0,3.733333,2013-04-01 18:53:59
3,00:00:13,u45,1364848920,in[baker-berry],13.0,0.216667,2013-04-01 20:42:00
4,00:00:40,u45,1364848920,in[baker-berry],40.0,0.666667,2013-04-01 20:42:00
...,...,...,...,...,...,...,...
11881,00:01:39,u49,1366374288,in[baker-berry],99.0,1.650000,2013-04-19 12:24:48
11882,00:04:48,u49,1366383400,in[kemeny],288.0,4.800000,2013-04-19 14:56:40
11883,01:17:51,u49,1366384317,in[cummings],4671.0,77.850000,2013-04-19 15:11:57
11884,08:49:00,u49,1366393411,in[sudikoff],31740.0,529.000000,2013-04-19 17:43:31


In [16]:
#The next step of my analysis consisted of actually finding the class attendance rates.
#I devised a methodology for this, and it involved finding the number of students in every class and as well as figuring out what days of the week those classes occured.
#My goal was to then find the number of expected attendances and then compare the number of actual attendances using the time deltas dataframe from before.
#From this, I can find class attendance as a percentage and see how it changes over the semester and incorporate it into our ML model


In [17]:
#This data looks at classes that students are taking over the semester.
class_data = pd.read_csv("class_data.csv")
class_data

,u00,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,u01,ENGS 069,ENGS 022,ANTH 012,NaN
1,u02,COSC 077,COSC 098,COSC 065,NaN
2,u03,COSC 057,COSC 065,NaN,NaN
3,u04,COSC 065,NaN,NaN,NaN
4,u05,COSC 050,PSYC 028,COSC 065,NaN
5,u07,COSC 077,COSC 060,COSC 065,NaN
6,u08,CHIN 062,COSC 089 1,COSC 065,NaN
7,u09,ANTH 050,COSC 065,COSC 099,NaN
8,u10,COSC 050,BIOL 004,COSC 065,NaN
9,u12,COSC 089 1,COSC 050,TUCK 003,COSC 065


In [18]:
#This data looks at classes that students are taking over the semester.
class_data = pd.read_csv("class_data.csv")

#To find the number of students in each class, I used the pd.value_counts function to do so.
def num_students_each_class(data):
    data = data.apply(pd.value_counts)
    data = data.head(44)
    #I created a column of the number of students in every class with the rows representing a class.
    data['Num Students in class'] = data.sum(axis=1)
    return data
class_data = num_students_each_class(class_data)

#The class_data dataframe had some irrelevant columns, which I deleted with the following code.
del class_data['u00'], class_data['Unnamed: 1'], class_data['Unnamed: 2'], class_data['Unnamed: 3'], class_data['Unnamed: 4']

#This dataframe has the number of students in each class
class_data

,Num Students in class
ANTH 012,1.0
ANTH 050,1.0
BIOL 004,1.0
BIOL 006,1.0
CHIN 033,1.0
CHIN 062,1.0
COSC 007,1.0
COSC 020,1.0
COSC 027,2.0
COSC 050,9.0


In [19]:
#With this code, I creaated a new dataframe from the class_info dataframe, but this dataframe has the classes and periods as columns instead of rows. 
def get_class_list_prelim_periods(data):
    list_classes = list(data.columns)
    periods_data = list(data.iloc[1])
    dataframe = pd.DataFrame({'Class': list_classes, 'periods': periods_data})
    return dataframe
class_info_df = get_class_list_prelim_periods(class_info)

#Since some classes appeared in the class dataframe while others did not appear in the class info dataframe, I found the common classes amongst both dataframes using this function.
def get_final_class_list(data1, data2):
    list_classes_prelim = list(data1.index)
    class_list = list(set(list_classes_prelim).intersection(list(data2.Class)))
    return class_list
class_list = get_final_class_list(class_data, class_info_df)
periods_data = list(class_info.iloc[1])

In [20]:
#Once I had the common classes shared between both dataframes, I merged them together.
def get_final_class_data(data1, data2, list1):
    data2 = data2[data2.Class.isin(list1)]
    data1 = data1[data1.index.isin(list1)]
    periods = list(data2.periods)
    data1['periods'] = periods
    data1["Class"] = data1.index
    return data1
class_data = get_final_class_data(class_data, class_info_df, class_list)

#This final dataframe has information for class type, number of students in each class, and when the class occurs
class_data

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


,Num Students in class,periods,Class
ANTH 012,1.0,"[{'day': 1, 'end': '12:20', 'start': '11:15'},...",ANTH 012
BIOL 004,1.0,"[{'day': 2, 'end': '11:50', 'start': '10:00'},...",BIOL 004
BIOL 006,1.0,"[{'day': 2, 'end': '11:50', 'start': '10:00'},...",BIOL 006
CHIN 033,1.0,"[{'day': 1, 'end': '09:50', 'start': '09:00'},...",CHIN 033
CHIN 062,1.0,"[{'day': 2, 'end': '17:50', 'start': '16:00'},...",CHIN 062
COSC 007,1.0,"[{'day': 1, 'end': '14:50', 'start': '13:45'},...",COSC 007
COSC 020,1.0,"[{'day': 2, 'end': '15:50', 'start': '14:00'},...",COSC 020
COSC 027,2.0,"[{'day': 1, 'end': '16:50', 'start': '15:00'},...",COSC 027
COSC 050,9.0,"[{'day': 1, 'end': '13:35', 'start': '12:30'},...",COSC 050
COSC 057,1.0,"[{'day': 1, 'end': '14:50', 'start': '13:45'},...",COSC 057


In [21]:
#This function returns a dataframe with two columns: Days, representing weekdays from 1-5 (Mon-Fri) and Class.
#The dataframe that is returned has the days in which each class takes place. 
#However, each individual day has its own row, with the corresponding class.
def get_class_days(data):
    lst_days = []
    lst_class = []
    for i in list(range(0, len(data['periods']))):
        for j in data['periods'][i]:
            try:
                lst_days.append(j['day'])
                lst_class.append(data.Class[i])
            except:
                lst_days.append('Error')
    df_class_days = pd.DataFrame({'Days': lst_days, 'Class': lst_class})
    return df_class_days
df_class_days = get_class_days(class_data)
#df_class_days

In [22]:
#This dataframe improves upon the previous dataframe by allocating all of the days in which each class ocurrs into a single row.
def get_class_days_array(data):
    group = data.groupby('Class')
    df2 = group.apply(lambda x: x['Days'].unique())
    df3 = pd.DataFrame({'Days': list(df2), 'Class': class_data.Class})
    df_class_days_final = df3.reset_index()
    return df_class_days_final
df_class_days_final = get_class_days_array(df_class_days)
#This dataframe has a column of arrays of numbers from 1-5 representing the days of Mon-Fri.
#Each array represents the days of the week in which the corresponding class takes place.
df_class_days_final.head()

,index,Days,Class
0,ANTH 012,"[1, 3, 5, 2]",ANTH 012
1,BIOL 004,"[2, 4, 3]",BIOL 004
2,BIOL 006,"[2, 4, 3]",BIOL 006
3,CHIN 033,"[1, 2, 4, 5, 3]",CHIN 033
4,CHIN 062,"[2, 4, 1]",CHIN 062


In [23]:
class_2 = pd.read_csv('class_data.csv')

In [24]:
#This function will be used to make a new column in the class_2 dataframe. 
#This new column will have arrays of the classes that each student is taking over the semester.
def class_p(class_series): 
    class_series = class_series.dropna()
    class_list = []
    for class_4 in class_series.values: 
        if class_4[0] == 'u': 
            continue
        else: 
            class_list.append(class_4)
    return class_list
class_2['total'] = class_2.apply(func = class_p, axis = 1)

In [25]:
class_2.head()

,u00,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,total
0,u01,ENGS 069,ENGS 022,ANTH 012,NaN,"[ENGS 069, ENGS 022, ANTH 012]"
1,u02,COSC 077,COSC 098,COSC 065,NaN,"[COSC 077, COSC 098, COSC 065]"
2,u03,COSC 057,COSC 065,NaN,NaN,"[COSC 057, COSC 065]"
3,u04,COSC 065,NaN,NaN,NaN,[COSC 065]
4,u05,COSC 050,PSYC 028,COSC 065,NaN,"[COSC 050, PSYC 028, COSC 065]"


In [26]:
#This dataframe adds on a new column of the days for each class.
def match_class_days(class_list):
    days = []
    for class_4 in class_list: 
        class_days = df_class_days_final[df_class_days_final.Class == class_4]['Days']       
        days.extend(class_days)
    return days
class_2['days'] = class_2['total'].apply(match_class_days)
class_2

,u00,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,total,days
0,u01,ENGS 069,ENGS 022,ANTH 012,NaN,"[ENGS 069, ENGS 022, ANTH 012]","[[1, 3, 5, 4], [1, 3, 5, 4], [1, 3, 5, 2]]"
1,u02,COSC 077,COSC 098,COSC 065,NaN,"[COSC 077, COSC 098, COSC 065]","[[2, 4, 3], [1, 3, 5, 4]]"
2,u03,COSC 057,COSC 065,NaN,NaN,"[COSC 057, COSC 065]","[[1, 3, 5, 4], [1, 3, 5, 4]]"
3,u04,COSC 065,NaN,NaN,NaN,[COSC 065],"[[1, 3, 5, 4]]"
4,u05,COSC 050,PSYC 028,COSC 065,NaN,"[COSC 050, PSYC 028, COSC 065]","[[1, 3, 5, 2], [1, 3, 5, 4], [1, 3, 5, 4]]"
5,u07,COSC 077,COSC 060,COSC 065,NaN,"[COSC 077, COSC 060, COSC 065]","[[2, 4, 3], [1, 3, 5, 4], [1, 3, 5, 4]]"
6,u08,CHIN 062,COSC 089 1,COSC 065,NaN,"[CHIN 062, COSC 089 1, COSC 065]","[[2, 4, 1], [1, 3, 5, 2], [1, 3, 5, 4]]"
7,u09,ANTH 050,COSC 065,COSC 099,NaN,"[ANTH 050, COSC 065, COSC 099]","[[1, 3, 5, 4]]"
8,u10,COSC 050,BIOL 004,COSC 065,NaN,"[COSC 050, BIOL 004, COSC 065]","[[1, 3, 5, 2], [2, 4, 3], [1, 3, 5, 4]]"
9,u12,COSC 089 1,COSC 050,TUCK 003,COSC 065,"[COSC 089 1, COSC 050, TUCK 003, COSC 065]","[[1, 3, 5, 2], [1, 3, 5, 2], [2, 4, 3], [1, 3,..."


In [27]:
#This function returns five new columns and adds them to the class_2 dataframe from before. 
#Each of the five columns represent the number of classes (and therefore number of attendances) each student has (should have) per day depending on the day of the week.
def get_classes_per_day(data):
    mon=[]
    tues=[]
    wed=[]
    thurs=[]
    fri=[]
    user_list = []
    list_arrays = []
    counter = 0
    for i in list(range(0, 48)):
    #print(i)
        for j in data.days[i]:
            mon.append(np.count_nonzero(j == 1))
            tues.append(np.count_nonzero(j == 2))
            wed.append(np.count_nonzero(j == 3))
            thurs.append(np.count_nonzero(j == 4))
            fri.append(np.count_nonzero(j == 5))
            user_list.append(data.u00[i])

    data = data[data['u00'] != 'u56']
    data = data[data['u00'] != 'u36']
    data = data[data['u00'] != 'u39']
    days_classes = pd.DataFrame({'uid': user_list, 'Monday': mon, 'Tuesday': tues, 'Wednesday': wed, 'Thursday': thurs, 'Friday': fri})
    num_classes_monday = (days_classes.groupby('uid').Monday.sum())
    num_classes_tuesday = (days_classes.groupby('uid').Tuesday.sum())
    num_classes_wednesday = (days_classes.groupby('uid').Wednesday.sum())
    num_classes_thursday = (days_classes.groupby('uid').Thursday.sum())
    num_classes_friday = (days_classes.groupby('uid').Friday.sum())
    data['Monday'] = list(num_classes_monday)
    data['Tuesday'] = list(num_classes_tuesday)
    data['Wednesday'] = list(num_classes_wednesday)
    data['Thursday'] = list(num_classes_thursday)
    data['Friday'] = list(num_classes_friday)
    return data
class_2 = get_classes_per_day(class_2)

In [28]:
class_2

,u00,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,total,days,Monday,Tuesday,Wednesday,Thursday,Friday
0,u01,ENGS 069,ENGS 022,ANTH 012,NaN,"[ENGS 069, ENGS 022, ANTH 012]","[[1, 3, 5, 4], [1, 3, 5, 4], [1, 3, 5, 2]]",3,1,3,2,3
1,u02,COSC 077,COSC 098,COSC 065,NaN,"[COSC 077, COSC 098, COSC 065]","[[2, 4, 3], [1, 3, 5, 4]]",1,1,2,2,1
2,u03,COSC 057,COSC 065,NaN,NaN,"[COSC 057, COSC 065]","[[1, 3, 5, 4], [1, 3, 5, 4]]",2,0,2,2,2
3,u04,COSC 065,NaN,NaN,NaN,[COSC 065],"[[1, 3, 5, 4]]",1,0,1,1,1
4,u05,COSC 050,PSYC 028,COSC 065,NaN,"[COSC 050, PSYC 028, COSC 065]","[[1, 3, 5, 2], [1, 3, 5, 4], [1, 3, 5, 4]]",3,1,3,2,3
5,u07,COSC 077,COSC 060,COSC 065,NaN,"[COSC 077, COSC 060, COSC 065]","[[2, 4, 3], [1, 3, 5, 4], [1, 3, 5, 4]]",2,1,3,3,2
6,u08,CHIN 062,COSC 089 1,COSC 065,NaN,"[CHIN 062, COSC 089 1, COSC 065]","[[2, 4, 1], [1, 3, 5, 2], [1, 3, 5, 4]]",3,2,2,2,2
7,u09,ANTH 050,COSC 065,COSC 099,NaN,"[ANTH 050, COSC 065, COSC 099]","[[1, 3, 5, 4]]",1,0,1,1,1
8,u10,COSC 050,BIOL 004,COSC 065,NaN,"[COSC 050, BIOL 004, COSC 065]","[[1, 3, 5, 2], [2, 4, 3], [1, 3, 5, 4]]",2,2,3,2,2
9,u12,COSC 089 1,COSC 050,TUCK 003,COSC 065,"[COSC 089 1, COSC 050, TUCK 003, COSC 065]","[[1, 3, 5, 2], [1, 3, 5, 2], [2, 4, 3], [1, 3,...",3,3,4,2,3


In [29]:
#This data frame has the start and end times for each class
#I also added two new columns, Start_low and End_high since students arrive early to class, but can also leave late.
#This creates a range for start and end times.
def get_start_end(data): #data is class_data
    lst_start_time = [] 
    lst_end_time = []
    lst_days = []
    lst_class = []
    for i in list(range(0, len(data['periods']))):
        for j in data['periods'][i]:
            try:
                lst_start_time.append(j['start'])
                lst_end_time.append(j['end'])
                lst_days.append(j['day'])
                lst_class.append(data.Class[i])
            except:
                lst_days.append('Error')
    df_class_times = pd.DataFrame({'Start': lst_start_time, 'End': lst_end_time,}) #'Days': lst_days, 'Class': lst_class})
    df_class_times = df_class_times.drop_duplicates()
    df_class_times["Start"] = pd.to_datetime(df_class_times['Start'])
    df_class_times['Start_low'] = df_class_times['Start'] - datetime.timedelta(minutes=20)
    df_class_times['End'] = pd.to_datetime(df_class_times['End'])
    df_class_times['End_high'] = df_class_times['End'] + datetime.timedelta(minutes=20)
    return df_class_times
df_class_times = get_start_end(class_data)
#This dataframe has start and end times, as well as low and high estimates, respectively.
df_class_times

,Start,End,Start_low,End_high
0,2020-07-11 11:15:00,2020-07-11 12:20:00,2020-07-11 10:55:00,2020-07-11 12:40:00
3,2020-07-11 12:00:00,2020-07-11 12:50:00,2020-07-11 11:40:00,2020-07-11 13:10:00
4,2020-07-11 10:00:00,2020-07-11 11:50:00,2020-07-11 09:40:00,2020-07-11 12:10:00
6,2020-07-11 15:00:00,2020-07-11 15:50:00,2020-07-11 14:40:00,2020-07-11 16:10:00
10,2020-07-11 09:00:00,2020-07-11 09:50:00,2020-07-11 08:40:00,2020-07-11 10:10:00
15,2020-07-11 16:00:00,2020-07-11 17:50:00,2020-07-11 15:40:00,2020-07-11 18:10:00
17,2020-07-11 17:00:00,2020-07-11 17:50:00,2020-07-11 16:40:00,2020-07-11 18:10:00
18,2020-07-11 13:45:00,2020-07-11 14:50:00,2020-07-11 13:25:00,2020-07-11 15:10:00
21,2020-07-11 13:00:00,2020-07-11 13:50:00,2020-07-11 12:40:00,2020-07-11 14:10:00
22,2020-07-11 14:00:00,2020-07-11 15:50:00,2020-07-11 13:40:00,2020-07-11 16:10:00


In [30]:
#Since the columns of the previous dataframe also had the date (which is the day when you run the code), and since we want just the hours, minutes and seconds, this code extracts that info.
df_class_times['Start']
df_class_times['End']
def get_time_format(times1, times2, times3, times4, data):
    start_times = []
    start_times_low = []
    end_times_high = []
    end_times = []
    for i in times1:
        start_times.append(i.strftime("%H:%M:%S"))
    for i in times2:
        start_times_low.append(i.strftime("%H:%M:%S"))
    for i in times3:
        end_times.append(i.strftime("%H:%M:%S"))
    for i in times4:
        end_times_high.append(i.strftime("%H:%M:%S"))
    data['Start'] = start_times
    data['End'] = end_times
    data['Start_low'] = start_times_low
    data['End_high'] = end_times_high
    data['Start Range'] = list(zip(data['Start_low'], data['Start']))
    data['End Range'] = list(zip(data['End'], data['End_high']))
    return data
#This dataframe has the class start, start_low, end, end_high, as well as ranges of times.
#The ranges of times are tuples of (start_low, start) and (end, end_high)
df_class_times_final = get_time_format(df_class_times['Start'], df_class_times['Start_low'], df_class_times['End'], df_class_times['End_high'], df_class_times)

In [31]:
#df_class_times_final['Start Range'] = list(zip(df_class_times['Start_low'], df_class_times['Start']))
#df_class_times_final['End Range'] = list(zip(df_class_times['End'], df_class_times['End_high']))
df_class_times_final

,Start,End,Start_low,End_high,Start Range,End Range
0,11:15:00,12:20:00,10:55:00,12:40:00,"(10:55:00, 11:15:00)","(12:20:00, 12:40:00)"
3,12:00:00,12:50:00,11:40:00,13:10:00,"(11:40:00, 12:00:00)","(12:50:00, 13:10:00)"
4,10:00:00,11:50:00,09:40:00,12:10:00,"(09:40:00, 10:00:00)","(11:50:00, 12:10:00)"
6,15:00:00,15:50:00,14:40:00,16:10:00,"(14:40:00, 15:00:00)","(15:50:00, 16:10:00)"
10,09:00:00,09:50:00,08:40:00,10:10:00,"(08:40:00, 09:00:00)","(09:50:00, 10:10:00)"
15,16:00:00,17:50:00,15:40:00,18:10:00,"(15:40:00, 16:00:00)","(17:50:00, 18:10:00)"
17,17:00:00,17:50:00,16:40:00,18:10:00,"(16:40:00, 17:00:00)","(17:50:00, 18:10:00)"
18,13:45:00,14:50:00,13:25:00,15:10:00,"(13:25:00, 13:45:00)","(14:50:00, 15:10:00)"
21,13:00:00,13:50:00,12:40:00,14:10:00,"(12:40:00, 13:00:00)","(13:50:00, 14:10:00)"
22,14:00:00,15:50:00,13:40:00,16:10:00,"(13:40:00, 14:00:00)","(15:50:00, 16:10:00)"


In [33]:
#This function returns the start and end time for each span of time a student was in a particular location
#The start and end times are also represented in hour form too.
def get_days(data):
    days_of_classes = list(range(0,5))
    data['Day'] = data['start time in regular notation'].dt.dayofyear
    data['Day of Week'] = data['start time in regular notation'].dt.dayofweek
    data = data[data['Day of Week'].isin(days_of_classes)]
    data['minutes_spent'] = data['Time Delta (in minutes)'].astype(int)
    data['seconds_spent'] = data['Time Delta (in seconds)'].astype(int)
    data['end time'] = data['start time'] + data['seconds_spent']
    data['start_hour'] = data['start time in regular notation'].dt.hour
    data['end time in regular notation'] = pd.to_datetime(data['end time'], unit = 's')
    data['end_hour'] = data['end time in regular notation'].dt.hour
    
    return data
df_new = get_days(df)
#The following code creates tuples of the stat and end times for further analysis
df_new['Start and End Times'] = list(zip(df_new['start time in regular notation'], df_new['end time in regular notation']))
#df_new

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: ht

In [34]:
df_new

,Time Delta,uid,start time,location,Time Delta (in seconds),Time Delta (in minutes),start time in regular notation,Day,Day of Week,minutes_spent,seconds_spent,end time,start_hour,end time in regular notation,end_hour,Start and End Times
0,00:23:36,u45,1364397661,in[sudikoff],1416.0,23.600000,2013-03-27 15:21:01,86,2,23,1416,1364399077,15,2013-03-27 15:44:37,15,"(2013-03-27 15:21:01, 2013-03-27 15:44:37)"
1,00:20:58,u45,1364839542,in[sudikoff],1258.0,20.966667,2013-04-01 18:05:42,91,0,20,1258,1364840800,18,2013-04-01 18:26:40,18,"(2013-04-01 18:05:42, 2013-04-01 18:26:40)"
2,00:03:44,u45,1364842439,in[lsb],224.0,3.733333,2013-04-01 18:53:59,91,0,3,224,1364842663,18,2013-04-01 18:57:43,18,"(2013-04-01 18:53:59, 2013-04-01 18:57:43)"
3,00:00:13,u45,1364848920,in[baker-berry],13.0,0.216667,2013-04-01 20:42:00,91,0,0,13,1364848933,20,2013-04-01 20:42:13,20,"(2013-04-01 20:42:00, 2013-04-01 20:42:13)"
4,00:00:40,u45,1364848920,in[baker-berry],40.0,0.666667,2013-04-01 20:42:00,91,0,0,40,1364848960,20,2013-04-01 20:42:40,20,"(2013-04-01 20:42:00, 2013-04-01 20:42:40)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11881,00:01:39,u49,1366374288,in[baker-berry],99.0,1.650000,2013-04-19 12:24:48,109,4,1,99,1366374387,12,2013-04-19 12:26:27,12,"(2013-04-19 12:24:48, 2013-04-19 12:26:27)"
11882,00:04:48,u49,1366383400,in[kemeny],288.0,4.800000,2013-04-19 14:56:40,109,4,4,288,1366383688,14,2013-04-19 15:01:28,15,"(2013-04-19 14:56:40, 2013-04-19 15:01:28)"
11883,01:17:51,u49,1366384317,in[cummings],4671.0,77.850000,2013-04-19 15:11:57,109,4,77,4671,1366388988,15,2013-04-19 16:29:48,16,"(2013-04-19 15:11:57, 2013-04-19 16:29:48)"
11884,08:49:00,u49,1366393411,in[sudikoff],31740.0,529.000000,2013-04-19 17:43:31,109,4,529,31740,1366425151,17,2013-04-20 02:32:31,2,"(2013-04-19 17:43:31, 2013-04-20 02:32:31)"


In [35]:
#From the SmartGPA paper, only times certain times spent in class locations counted towards class attendance.
#The times were atleast 90% of Dartmouth class times. I also added a buffer of 5 min for students that stayed after.
#The times were:
# 45-55 min, 58-70 min, and 99-115 min.
#This function searched through the previous dataframe to find only time deltas that matched these specifications.
def eligible_times(data):
    list_eligible_times = []
    list_eligible_uids = []
    list_eligible_locations = []
    list_eligible_days = []
    list_eligible_weekdays = []
    for i in data.index:
        minutes_spent = data.loc[i]['minutes_spent']
        if 45 <= minutes_spent <= 60:
            list_eligible_times.append(minutes_spent)
            list_eligible_uids.append(df.loc[i]['uid'])
            list_eligible_locations.append(df.loc[i]['location'])
            list_eligible_days.append(df.loc[i]['Day'])
            list_eligible_weekdays.append(df.loc[i]['Day of Week'])
        elif 58 <= minutes_spent <= 75:
            list_eligible_times.append(minutes_spent)
            list_eligible_uids.append(df.loc[i]['uid'])
            list_eligible_locations.append(df.loc[i]['location'])
            list_eligible_days.append(df.loc[i]['Day'])
            list_eligible_weekdays.append(df.loc[i]['Day of Week'])
        elif 99 <= minutes_spent <= 120:
            list_eligible_times.append(minutes_spent)
            list_eligible_uids.append(df.loc[i]['uid'])
            list_eligible_locations.append(df.loc[i]['location'])
            list_eligible_days.append(df.loc[i]['Day'])
            list_eligible_weekdays.append(df.loc[i]['Day of Week'])
    df_new = pd.DataFrame({'uid': list_eligible_uids, 'minutes spent': list_eligible_times, 'location': list_eligible_locations, 'Day': list_eligible_days, 'Day of Week': list_eligible_weekdays})
    return df_new
#This new dataframe contains only the time deltas that met the SmartGPA paper's certifications.
df_final_processing = eligible_times(df_new)

In [36]:
df_final_processing.tail(25)

,uid,minutes spent,location,Day,Day of Week
787,u16,104,in[baker-berry],148,1
788,u15,47,in[lsb],86,2
789,u15,53,in[baker-berry],87,3
790,u15,54,in[lsb],88,4
791,u15,54,in[lsb],98,0
792,u15,62,in[lsb],120,1
793,u15,53,in[lsb],130,4
794,u15,50,in[hopkins],133,0
795,u15,73,in[sudikoff],147,0
796,u15,45,in[lsb],149,2


In [37]:
#Since I have a dataframe of time spans in class locations within the periods in which classes take place, I can infer class attendance rates.
#The number of rows for each student for which the day is the same is equivalent to the number of class attendances that student had for that particular day.
def attendance_counter(data):
    attendance_num = 1
    #counter = 0
    list_attendance_counter = []
    list_id = []
    for i in range(0, len(data)):
        #if data.location[i] == data.location[i+1]:
        try: 
            if data.Day[i] == data.Day[i+1]:
            #data.index[i] == (data.index[i+1] - 1)
                attendance_num += 1
                #uid = data.uid[i]
            else:
                list_attendance_counter.append(attendance_num)
                attendance_num = 1
                #list_id.append(uid)
        except:
            list_attendance_counter.append(attendance_num)
            attendance_num = 1
            #list_id.append(uid)
    return list_attendance_counter

#This list has all of the attendances per day (# of consecutive rows where day is the same)
list_attendances = attendance_counter(df_final_processing)

In [58]:
#To be able to track user id, day and day of week alongside the number of class attendances, I had to use this function:
#This function is based off a formula I found that can find the index for the following data:
    #user id, day and day of week
def get_sums(list1):
    list_sums_b = []
    for i in range(0,len(list1)+1):
        new_list = list1[:i]
        sums = sum(new_list)
        list_sums_b.append(sums)
    return list_sums_b
list_attendance_shifts = get_sums(list_attendances)
list_new = list_attendance_shifts

def get_index_list(list1):
    list_finale = []
    for i in list1:
        if i <= 811:
            list_finale.append(i)
    return list_finale
list_finale = get_index_list(list_new)

In [68]:
#This function creates a new dataframe that consists of, for everyday, the user id, day and day of week parameters.
#I was able to create a new column in this dataframe that consists of the number of class attendances per student per day.
def get_days(data, list1):
    days = []
    uid = []
    weekday = []
    for i in range(0, len(list1)):
        weekday.append(data['Day of Week'][list1[i]])
        uid.append(data.uid[list1[i]])
        days.append(data.Day[list1[i]])
        df1 = pd.DataFrame({'uid': uid, 'Day': days, 'Weekday': weekday})#'Num Attendances': list_attendances})
    return df1
df_finale = get_days(df_final_processing, list_finale)
df_finale['Num Attendances'] = list_attendances

In [70]:
#This function assigns the name of the day for each day of week parameter in the previous dataframe.
def get_days(data):
    list_day_names = []
    for i in data.Weekday:
        if i == 0:
            list_day_names.append('Monday')
        elif i == 1:
            list_day_names.append('Tuesday')
        elif i == 2:
            list_day_names.append('Wednesday')
        elif i == 3:
            list_day_names.append('Thursday')
        else:
            list_day_names.append('Friday')
    data['Name'] = list_day_names
    return data
df_finale = get_days(df_finale)
df_finale

,uid,Day,Weekday,Num Attendances,Name
0,u45,92,1,1,Tuesday
1,u45,98,0,2,Monday
2,u45,99,1,2,Tuesday
3,u45,102,4,1,Friday
4,u45,105,0,1,Monday
...,...,...,...,...,...
529,u49,95,4,4,Friday
530,u49,100,2,2,Wednesday
531,u49,101,3,4,Thursday
532,u49,102,4,2,Friday


In [71]:
class_2 = class_2.reset_index()

In [72]:
#This function removes student data for which there is no class data.
def remove_students_no_data(data):
    data = data[data['uid'] != 'u56']
    data = data[data['uid'] != 'u36']
    data = data[data['uid'] != 'u39']
    data = data[data['uid'] != 'u00']
    return data
df_finale = remove_students_no_data(df_finale)
df_finale = df_finale.reset_index()
df_finale

,index,uid,Day,Weekday,Num Attendances,Name
0,0,u45,92,1,1,Tuesday
1,1,u45,98,0,2,Monday
2,2,u45,99,1,2,Tuesday
3,3,u45,102,4,1,Friday
4,4,u45,105,0,1,Monday
...,...,...,...,...,...,...
478,529,u49,95,4,4,Friday
479,530,u49,100,2,2,Wednesday
480,531,u49,101,3,4,Thursday
481,532,u49,102,4,2,Friday


In [73]:
#For each student in the dataframe, there is data on the number of class attendances per day.
#There isn't data on number of classes each student had that day.
#This depends on the number of classes they have for that particular day of the week (i.e. Monday, etc.)
#We already have data for this, and this function adds that data to the dataframe.

def match_classes(data):
    num_classes = []
    for i in range(0,len(data)):
        if data.Name[i] == 'Monday':
            for j in range(0, len(class_2)):
                if class_2.u00[j] == data.uid[i]:
                    num_classes.append(class_2['Monday'][j])
        elif data.Name[i] == 'Tuesday':
            for j in range(0, len(class_2)):
                if class_2.u00[j] == data.uid[i]:
                    num_classes.append(class_2['Tuesday'][j])
        elif data.Name[i] == 'Wednesday':
            for j in range(0, len(class_2)):
                if class_2.u00[j] == data.uid[i]:
                    num_classes.append(class_2['Wednesday'][j])
        elif data.Name[i] == 'Thursday':
            for j in range(0, len(class_2)):
                if class_2.u00[j] == data.uid[i]:
                    num_classes.append(class_2['Thursday'][j])
        else:
            for j in range(0, len(class_2)):
                if class_2.u00[j] == data.uid[i]:
                    num_classes.append(class_2['Friday'][j])
        #data['Num Classes on Day'] = num_classes
    return num_classes
num_classes_list = match_classes(df_finale)

In [74]:
#This function adds the data on the number of classes each student had for a particular data point in the dataframe to a new column in the dataframe.
#So now we have data on #of class attendances and #of classes per day.
#The final step is to find the class attendance rates, which is simply a ratio.
def get_class_attend_rates(data, list1):
    data['Num Classes on Day'] = list1 
    data['Attendance Rate'] = data['Num Attendances']/data['Num Classes on Day']
    return data

#This dataframe has class attendance rate information
df_finale = get_class_attend_rates(df_finale, num_classes_list)

In [75]:
#The last step was to find the actual rates.
#Students often attend events held at class location facilities.
#Thus, for some students, the attendance rates are higher than 1.
#This function finds the actual class attendance rates by resetting the rates greater than 1 to 1

def get_actual_rates(data):
    list_rates = []
    for i in data['Attendance Rate']:
        if i <= 1.0:
            list_rates.append(i)
        else:
            list_rates.append(1.0)
    data['Actual Rates'] = list_rates
    return data

#This is the final dataframe with the final class attendance rate data
df_class_attendance_rates = get_actual_rates(df_finale)

In [76]:
df_class_attendance_rates

,index,uid,Day,Weekday,Num Attendances,Name,Num Classes on Day,Attendance Rate,Actual Rates
0,0,u45,92,1,1,Tuesday,1,1.000000,1.000000
1,1,u45,98,0,2,Monday,1,2.000000,1.000000
2,2,u45,99,1,2,Tuesday,1,2.000000,1.000000
3,3,u45,102,4,1,Friday,1,1.000000,1.000000
4,4,u45,105,0,1,Monday,1,1.000000,1.000000
...,...,...,...,...,...,...,...,...,...
478,529,u49,95,4,4,Friday,3,1.333333,1.000000
479,530,u49,100,2,2,Wednesday,3,0.666667,0.666667
480,531,u49,101,3,4,Thursday,2,2.000000,1.000000
481,532,u49,102,4,2,Friday,3,0.666667,0.666667


In [49]:
df_class_attendance_rates.to_csv('Digital Phenotyping\Class Attendance.csv')

In [ ]:
#It was great to be able to finally get this data.
#Michael was able to add this data to his GPA predictor model, which was enhanced by this data.